In [2]:
from core.llm_client import LLMClient
import os

model_type = os.getenv("MODEL_TYPE", "openai")
# model_type = os.getenv("", "gemini")
llm_client = LLMClient(model_type=model_type)

model_type

'openai'

In [3]:
from prompts.planner import SYSTEM_PROMPT
from utility import load_json

current directory :  D:\Study\Programs\TSAI\EAG_V1_CAPESTONE\dev_docs\tests


In [4]:
# Test Case 1 – First Message (new workflow)
user_prompt_1= str({
  "CURRENT_MESSAGE": "I want to buy a laptop",
  "PAST_5_TURNS": [],
  "SESSION_WORKSTREAMS": []
    })
response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_1)
print(response)

{
  "phase": "DISCOVERY",
  "phase_confidence": 0.95,
  "entities": {
    "subcategory": ["laptop"],
    "order_id": []
  },
  "decision": {
    "new_workstreams": [
      {
        "phase": "DISCOVERY",
        "target": {
          "subcategory": "laptop",
          "order_id": null
        }
      }
    ],
    "active_workflow_phase": "UNKNOWN",
    "active_workflow_continuity": "SWITCH",
    "focus_workstream_id": null
  },
  "reason": "The user is initiating a new workflow to discover laptops."
}


In [15]:
# Test Case 2 – Switch (pause old, add new)
user_prompt_2= str({
      "CURRENT_MESSAGE": "Also show me smartphones",
      "PAST_3_TURNS": [
        { "user": "I want to buy a laptop", "agent": "Here are some options" }
      ],
      "LAST_INTENT": "DISCOVERY"
    })
response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_2)
print(response)

```json
{
  "current_turn": {
    "intent": "DISCOVERY",
    "confidence": 0.95,
    "entities": {
      "products": [
        {
          "category": "electronics",
          "subcategory": "smartphone"
        }
      ]
    },
    "referenced_entities": [],
    "reasoning": "User explicitly adds smartphones to their product search."
  },
  "continuity": {
    "continuity_type": "SWITCH",
    "confidence": 0.9,
    "reasoning": "User was in a laptop DISCOVERY workflow and introduced smartphones as a new target.",
    "workstream_decision": {
      "new_workstreams": [
        {
          "type": "DISCOVERY",
          "target": {
            "category": "electronics",
            "subcategory": "smartphone"
          }
        }
      ],
      "existing_workflow_status": "PAUSE"
    },
    "clarification_message": "You are now looking at laptops and smartphones. Do you want me to create a new workflow for smartphones?"
  }
}
```


In [14]:
# Test Case 3 – Referential continuation
user_prompt_3= str({
      "CURRENT_MESSAGE": "The third in the list looks good",
      "PAST_3_TURNS": [
        { "user": "Show me laptops", "agent": "1. Dell, 2. HP, 3. Lenovo" }
      ],
      "LAST_INTENT": "DISCOVERY"
    })
response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_3)
print(response)

```json
{
  "current_turn": {
    "intent": "DISCOVERY",
    "confidence": 0.8,
    "entities": {
      "products": []
    },
    "referenced_entities": [
      {
        "category": "electronics",
        "subcategory": "laptop",
        "source": "agent_list_item_3"
      }
    ],
    "reasoning": "User selects the third option (Lenovo laptop) from the agent’s list."
  },
  "continuity": {
    "continuity_type": "CONTINUATION",
    "confidence": 0.9,
    "reasoning": "User is continuing the ongoing laptop DISCOVERY by selecting a shown option.",
    "workstream_decision": {
      "new_workstreams": [],
      "existing_workflow_status": "CONTINUE"
    },
    "clarification_message": null
  }
}
```


In [13]:
# Test Case 4 – Switch to Return (abandon old)

user_prompt_4= str({
      "CURRENT_MESSAGE": "I need to return my headphones",
      "PAST_3_TURNS": [
        { "user": "Show me laptops", "agent": "Here are some Dell and HP options" }
      ],
      "LAST_INTENT": "DISCOVERY"
    })
response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_4)
print(response)

```json
{
  "current_turn": {
    "intent": "RETURN",
    "confidence": 0.9,
    "entities": {
      "products": [
        {
          "category": "electronics",
          "subcategory": "headphones"
        }
      ]
    },
    "referenced_entities": [],
    "reasoning": "The user explicitly states their intention to return headphones."
  },
  "continuity": {
    "continuity_type": "SWITCH",
    "confidence": 0.9,
    "reasoning": "The user switches from a laptop discovery workflow to a headphone return workflow.",
    "workstream_decision": {
      "new_workstreams": [
        {
          "type": "RETURN",
          "target": {
            "category": "electronics",
            "subcategory": "headphones"
          }
        }
      ],
      "existing_workflow_status": "PAUSE"
    },
    "clarification_message": "Okay, let's start the return process for your headphones. I'll pause the laptop search. Is that correct?"
  }
}
```


In [12]:
# Test Case 5 – Unclear (irrelevant message)
user_prompt_5= str({
      "CURRENT_MESSAGE": "What a    bout the weather today?",
      "PAST_3_TURNS": [
        { "user": "I want to buy a laptop", "agent": "Here are some Dell and HP options" }
      ],
      "LAST_INTENT": "DISCOVERY"
    })
response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_5)
print(response)

```json
{
  "current_turn": {
    "intent": "CHITCHAT",
    "confidence": 0.8,
    "entities": {
      "products": []
    },
    "referenced_entities": [],
    "reasoning": "The message is about the weather, indicating a casual conversation unrelated to e-commerce."
  },
  "continuity": {
    "continuity_type": "UNCLEAR",
    "confidence": 0.95,
    "reasoning": "The query is irrelevant to the ongoing DISCOVERY workflow for laptops.",
    "workstream_decision": {
      "new_workstreams": [],
      "existing_workflow_status": "UNCHANGED"
    },
    "clarification_message": "That seems unrelated to shopping. Do you want to continue with laptops?"
  }
}
```
